In [2]:
%load_ext autoreload
%autoreload 2

from sklearn.pipeline import Pipeline

from transformers import TextCleaner, TextFeatureExtractor
from helper import configure_logging
from markov import lexical_freq, pos_emission_prob, pos_markov_chain, vocabulary, markov_to_graph, \
beam_search, LowerMarkovifyText
from corpus_helper import CorpusStreamer, CorpusFileHandler

configure_logging('markov.log', logging.DEBUG)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
mapping = lambda x: (x.tag_, #category
                     x.is_stop, #starts_with_vowel(x.text), #bool
                     x.i - x.sent.start, x.sent.end - x.i, len(x), lexical_freq(x.text)) #number

class TextParser(Pipeline):
    def __init__(self):
        super().__init__([
            ("TextCleaner",TextCleaner()),
            ("TextFeatureExtractor", TextFeatureExtractor(mapping))
        ])

NameError: name 'Pipeline' is not defined

In [ ]:
folder_path = '../DataAcquisition/data/'
for author in os.listdir(folder_path)[2:]:
    path = os.path.join(folder_path, author)
    print(path)
    chain, pos_chain = pos_markov_chain(path, state_size=3)
    emission_probs = pos_emission_prob(path)
    #g = markov_to_graph(pos_chain)

In [ ]:
folder_path = '../DataAcquisition/data/'
with open(os.path.join(folder_path, 'wells', 'parsed', 'word_mm.pkl'), 'r') as f:
    chain = LowerMarkovifyText.from_json(f.read())

In [ ]:
words = ['war', 'men', 'created']
probs = beam_search(chain, pos_chain, words, 
                    beam_size=10, 
                    word_trans_weight=1, 
                    emission_weight=1, 
                    context_weight=.2, 
                    eos_norm_weight=0, 
                    len_norm_weight=.05,
                    smoothing_prob=1e-6,
                    variable_length=True)

In [45]:
vocab = vocabulary(chain)
sortvocab = sorted(vocab, key=vocab.get, reverse=True)